# Train with RAPIDS on multiple GPUs

description: train with RAPIDS, cuML, cuDF, and dask on multiple V100s on the full airline dataset

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

In [ ]:
# training script
script_dir = "src"
script_name = "train.py"

# environment file
environment_file = "DOCKERFILE"

# azure ml settings
environment_name = "rapids-tutorial"
experiment_name = "rapids-airline-multi-tutorial"
compute_target = "gpu-V100-4"

In [ ]:
from azureml.core import ScriptRunConfig, Experiment, Environment, Dataset
from azureml.core.runconfig import MpiConfiguration

ds = Dataset.File.from_files(
    "https://airlinedataset.blob.core.windows.net/airline-10years/*"
)

arguments = [
    "--data_dir",
    ds.as_mount(),
    "--n_bins",
    32,
    "--compute",
    "multi-GPU",
    "--cv-folds",
    1,
]

env = Environment(environment_name)
env.docker.enabled = True
env.docker.base_image = None
env.docker.base_dockerfile = environment_file
env.python.user_managed_dependencies = True

src = ScriptRunConfig(
    source_directory=script_dir,
    script=script_name,
    arguments=arguments,
    environment=env,
    compute_target=compute_target,
    max_run_duration_seconds=1800,
)

run = Experiment(ws, experiment_name).submit(src)
run

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

In [ ]:
run.wait_for_completion(show_output=True)